**KASIFIKASI OUTPUT TARGET**

0.   BELOK KANAN TAJAM
1.   BELOK KIRI TAJAM
2.   BELOK KANAN SIKU
3.   BELOK KIRI SIKU



# Mounting to Drive

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(42)

from google.colab import drive
drive.mount('/content/drive')

csv_file_path = '/content/drive/MyDrive/Colab Notebooks/MLP-Learning-LineFollower/data/linefollower_dataset_transformed.csv'

try:
    simulated_data = pd.read_csv(csv_file_path)
    print("Data loaded successfully from CSV.")
    display(simulated_data.head())
    display(simulated_data['gerakan'].value_counts())
except FileNotFoundError:
    print(f"Error: CSV file not found at {csv_file_path}")
    print("Please make sure the file exists and the path is correct.")
    simulated_data = None 

# Training Data

In [ ]:
import tensorflow as tf
tf.random.set_seed(42)        # random data sebanyak 42 data
from sklearn.model_selection import train_test_split     
from tensorflow.keras.utils import to_categorical         

y = simulated_data['gerakan']
X = simulated_data.drop('gerakan', axis=1)

# Convert ke numpy array
X = X.values
y = y.values

# Train-Test Split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,       
    random_state=42,
    shuffle=True,   
    stratify=y      # agar seimbang jumlahnya
)

# One-hot encoding untuk label (encoder angka 0-3 ke biner agar tidak membaca lebih dari/kurang dari)
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

print("Train X:", X_train.shape)
print("Train y:", y_train_one_hot.shape)
print("Test X:", X_test.shape)
print("Test y:", y_test_one_hot.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(80, activation='relu', input_shape=(X.shape[1],)), 
    Dense(40, activation='relu'), 
    Dense(y_train_one_hot.shape[1], activation='softmax') # Output layer with softmax activation for multi-class classification 
])

# Compile the model Adam karena lebih cepat dan adaptif dengan kondisi proses training // SGD
# Adam = Teknik untuk memperbarui bobot saat training.
model.compile(optimizer='adam',   
              loss='categorical_crossentropy',      # karena di encoder ke biner, dihitung dari selisih probabilitas softmax terhadap one-hot label
              metrics=['accuracy'])

model.summary()

# banyaknya bias tergantung jumlah node
# (input_features * neurons) + bias
# (10 * 100) + 100 = 1,100

In [ ]:
# menghentikan training otomatis saat mulai overfitting
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=20,                  # Jika 20 epoch berturut-turut tidak ada peningkatan val_loss → hentikan training
    restore_best_weights=True     # Setelah training berhenti → bobot model dikembalikan ke kondisi epoch terbaik
)

history = model.fit(
    X_train, y_train_one_hot,
    epochs=200,
    batch_size=32,
    validation_data=(X_test, y_test_one_hot),    # gunakan data test sebagai validation
    callbacks=[early_stop]
)

In [ ]:
# BACA GRAFIK TRAINING
# Training loss turun, validation loss turun seiring → bagus
# Training loss terus turun, validation loss malah naik → overfitting
# Baik training & validation loss tinggi → underfitting

import matplotlib.pyplot as plt

# Plot Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot Accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_one_hot, verbose=0)
print(f"Model Loss: {loss:.4f}")
print(f"Model Accuracy: {accuracy:.4f}")

In [ ]:
from tensorflow.keras import Model

def check_dead_neurons(model, X_sample):
    print("\n Cek Neuron Mati di Setiap Layer (ReLU Layers):")

    if not model.built:
        model.build(input_shape=(None, X.shape[1]))
        print("Model built explicitly before checking dead neurons.")

    for i, layer in enumerate(model.layers):
        config = layer.get_config()
        if 'activation' in config and config['activation'] == 'relu':
            intermediate_model = Model(inputs=model.layers[0].input, outputs=layer.output)
            output_values = intermediate_model.predict(X_sample, verbose=0)
            
            dead_mask = np.all(output_values == 0, axis=0)
            dead_neurons = np.sum(dead_mask)

            total_neurons = output_values.shape[1]
            if total_neurons > 0:
                persen = (dead_neurons / total_neurons) * 100
                print(f"Layer {i} ({layer.name}) → {dead_neurons}/{total_neurons} neuron mati ({persen:.1f}%)")
            else:
                print(f"Layer {i} ({layer.name}) → No neurons found.")

check_dead_neurons(model, X_train[:100])

# Ambil Bobot dan Bias

In [ ]:
W1, b1 = model.layers[0].get_weights()
W2, b2 = model.layers[1].get_weights()
W3, b3 = model.layers[2].get_weights()

np.set_printoptions(suppress=True, precision=6)

# --- LAPISAN 1 (Input IR 10 ke 100 Neuron) ---
print(f"// --- LAPISAN 1: Bobot (W1) ---")
print(f"float W1[{W1.shape[0]}][{W1.shape[1]}] = {{")
for row in np.round(W1, 6):
    print("  {", ', '.join(map(str, row.tolist())), "},")
print("};")

print(f"\n// --- LAPISAN 1: Bias (b1) ---")
print(f"float b1[{b1.shape[0]}] = {{", ', '.join(map(str, np.round(b1, 6).tolist())), "};")

# --- LAPISAN 2 (100 Neuron ke 50 Neuron) ---
print(f"\n// --- LAPISAN 2: Bobot (W2) ---")
print(f"float W2[{W2.shape[0]}][{W2.shape[1]}] = {{")
for row in np.round(W2, 6):
    print("  {", ', '.join(map(str, row.tolist())), "},")
print("};")

print(f"\n// --- LAPISAN 2: Bias (b2) ---")
print(f"float b2[{b2.shape[0]}] = {{", ', '.join(map(str, np.round(b2, 6).tolist())), "};")

# --- LAPISAN 3 (50 Neuron ke 4 Neuron Output) ---
print(f"\n// --- LAPISAN 3: Bobot (W3) ---")
print(f"float W3[{W3.shape[0]}][{W3.shape[1]}] = {{")
for row in np.round(W3, 6):
    print("  {", ', '.join(map(str, row.tolist())), "},")
print("};")

print(f"\n// --- LAPISAN 3: Bias (b3) ---")
print(f"float b3[{b3.shape[0]}] = {{", ', '.join(map(str, np.round(b3, 6).tolist())), "};")


# Model Save

In [ ]:
model_filename = 'model_mlp.h5'
drive_path_model = '/content/drive/MyDrive/Colab Notebooks/MLP-Learning-LineFollower/models/'

model.save(drive_path_model + model_filename)

print(f"Model MLP berhasil disimpan sebagai: {model_filename}")